<a href="https://colab.research.google.com/github/timonpalm/TFCourse/blob/main/08_introduction_2_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in Tensorflow

# Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-8dfd6b95-4740-4dfe-1a8d-67aeba36e512)


## Get Helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-12-30 09:38:11--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-30 09:38:11 (24.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

Teh dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster)

See https://www.kaggle.com/competitions/nlp-getting-started/overview

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-12-30 09:38:17--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.207, 74.125.197.207, 74.125.135.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2023-12-30 09:38:17 (42.9 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text datset

In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
train_df.sample(n=5)

,id,keyword,location,text,target
6154,8780,siren,NaN,The real question is why is the tornado siren ...,1
6611,9467,terrorism,NaN,Pakistan's Supreme Court rules to allow milita...,1
3592,5131,fatal,"New South Wales, Australia",Man charged over fatal crash near Dubbo refuse...,1
2631,3774,destruction,NaN,@BlossomingLilac so destruction it seems. I se...,0
2330,3351,demolished,"Paterson, New Jersey",Three Homes Demolished in Unrecognized Arab Vi...,1


In [11]:
import random
random_index = random.randint(0, len(train_df)-5)
for _, text, target in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  print(f"Target: {target}", "(real disaster)" if target > 0  else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
I burned myself today on the oven ?? it was at 500 degrees ??

---

Target: 0 (not real disaster)
Text:
Inundated with employee holiday request paperwork? Our cloud based HR software @hronlinetweets will help simplify your staffing logistics.

---

Target: 1 (real disaster)
Text:
@FedPorn I feel your pain. Survivors will look back on this period as most absurd in human history. Satire indistinguishable from reality.

---

Target: 1 (real disaster)
Text:
This just-married Turkish couple gave 4000 Syrian refugees an incredible gift #WashingtonPost http://t.co/aRKmc7vclN

---

Target: 1 (real disaster)
Text:
Ina Buted Girl Crush??

---



### Split data into training and validation

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# Use train_test_split to split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [15]:
# Checl the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [16]:
# Check the first 10 sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Text Vectorization

In [20]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the deafult TextVerctorization
text_vectorizer = TextVectorization(max_tokens=10000, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int",
                                    output_sequence_length=None, # length of the sequences
                                    pad_to_max_tokens=True)

In [23]:
# Find the average number of tokens (words) in th training tweets
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [24]:
# Setup text vectorization varaibles
max_vocab_length = 10000 # max number of words in vocab
max_length = 15 # max length our sequences will be (how many words of the tweets will the model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [25]:
# Fit the text vectorizor to the train data
text_vectorizer.adapt(train_sentences)

In [26]:
# Create a sample sentence and tokinize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [27]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVecorized version:")
text_vectorizer([random_sentence])

Original text:
 @brookesddl I am traumatised the lil shit nearly hopped in the bloody shower with me        

Vecorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    8,  160,  412,    2, 1646,  235,  841,    1,    4,    2,
         261, 1593,   14,   31]])>

In [28]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding Layer

In [29]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                              output_dim=128,
                              input_length=max_length)

embedding

In [31]:
# Get a radnom sentence from the training set
random_sentence = random.choice(train_sentences)

print(f"Original text:\n {random_sentence}\
        \n\nEmbedding version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/SO81Ab3a1x        

Embedding version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00263817, -0.00954599, -0.00640699, ..., -0.04072858,
         -0.03832925, -0.00357586],
        [ 0.0268587 ,  0.02475299, -0.03932164, ..., -0.04929435,
          0.02073573, -0.03091126],
        [ 0.01040358,  0.04184468, -0.04894697, ...,  0.02416332,
          0.00458957,  0.01627502],
        ...,
        [ 0.01054329, -0.0386973 ,  0.00283483, ...,  0.00104351,
          0.03547553,  0.04627282],
        [-0.00345471, -0.02033439, -0.02761397, ...,  0.02454765,
          0.00997758,  0.00960523],
        [ 0.01836728, -0.02798926,  0.00704913, ..., -0.0416657 ,
         -0.04613042, -0.0261956 ]]], dtype=float32)>

In [33]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00263817, -0.00954599, -0.00640699, -0.0182365 ,  0.03242658,
         0.03815034, -0.02175026, -0.02120135,  0.02388327, -0.04702274,
         0.03332628, -0.00870073, -0.0291556 , -0.00556828, -0.02957385,
         0.03501984,  0.03389963,  0.00694323, -0.00290734,  0.04470685,
         0.03134367,  0.02127792,  0.01963894,  0.03203202,  0.03173871,
         0.02048495, -0.03081455,  0.00468993, -0.00129242,  0.04702783,
         0.02689283,  0.01124632, -0.00603582,  0.00822673,  0.00537422,
        -0.04722553,  0.02997083,  0.03511342, -0.00241008, -0.00782744,
        -0.03029385,  0.02084471, -0.04476796, -0.0041648 , -0.02048862,
         0.04116615,  0.01791677,  0.02793706,  0.03233541,  0.00127495,
        -0.02952348,  0.01600405, -0.04739172, -0.00259525, -0.01462334,
        -0.04485212,  0.01525593,  0.03954612, -0.02525046, -0.00227709,
         0.0223833 , -0.01313885, -0.0292596 , -0.03177358, -0.02807424,
  

## Modelling a text dataset (sunning a series of experiments)

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed Forward neural network (dense model)
* Model 2: LSTM (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

### Model 0: Getting a baseline

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [35]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [38]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])